<a href="https://colab.research.google.com/github/GarryCampbell/HousePrices/blob/main/HousePrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the dataset

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [3]:
! pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
%cd /content/gdrive/MyDrive/Datasets/"HousePrice"/

/content/gdrive/MyDrive/Datasets/1.HousePrice


In [12]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Datasets/HousePrice/"

In [13]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


# Check what is in the dataset

In [14]:
ls


house-prices-advanced-regression-techniques.zip  kaggle.json


# Unzip the dataset

Syntax - !unzip curr_path_to_file.zip -d destination_path


In [17]:
!unzip /content/gdrive/MyDrive/Datasets/HousePrice/house-prices-advanced-regression-techniques.zip -d /content/gdrive/MyDrive/Datasets/HousePrice/

Archive:  /content/gdrive/MyDrive/Datasets/HousePrice/house-prices-advanced-regression-techniques.zip
  inflating: /content/gdrive/MyDrive/Datasets/HousePrice/data_description.txt  
  inflating: /content/gdrive/MyDrive/Datasets/HousePrice/sample_submission.csv  
  inflating: /content/gdrive/MyDrive/Datasets/HousePrice/test.csv  
  inflating: /content/gdrive/MyDrive/Datasets/HousePrice/train.csv  


In [18]:
ls


data_description.txt                             sample_submission.csv
house-prices-advanced-regression-techniques.zip  test.csv
kaggle.json                                      train.csv


In [20]:
import pandas as pd

training_df = pd.read_csv("train.csv")
training_df.head()



,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
training_df.info()

In [ ]:
training_df.describe()

In [ ]:
training_df.columns


# Test 1 - Numbers only columns 

* Removing non-numerical columns
* Removing any columns containing nulls
* Using SKLearn to do linear regression
* 80:20 split of training.csv in the below work

In [ ]:
import numpy as np

training_df.select_dtypes(include=np.number).head()


In [40]:
from sklearn.model_selection import train_test_split

num_only_df = training_df.select_dtypes(include=np.number)
num_only_df = num_only_df.dropna(axis=1)

train, test = train_test_split(num_only_df, test_size=0.2)

In [41]:
# train
train_y_num_only_df = train["SalePrice"]
train_X_num_only_df = train.drop(columns="SalePrice")

# test
test_y_num_only_df = test["SalePrice"]
test_X_num_only_df = test.drop(columns="SalePrice")
test_X_num_only_df.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
719,720,20,9920,5,6,1969,1969,523,0,448,...,300,0,0,0,0,0,0,0,5,2006
576,577,50,6292,7,7,1928,1950,0,0,861,...,216,0,50,0,0,0,0,0,8,2009
1390,1391,20,9100,7,5,2000,2000,1400,0,125,...,541,219,36,0,0,0,0,0,9,2006
549,550,60,9375,7,5,2003,2004,0,0,912,...,615,182,182,0,0,0,0,0,11,2009
794,795,60,10832,7,5,1994,1996,0,0,712,...,409,143,46,0,0,0,0,500,10,2008


In [42]:
from sklearn.linear_model import LinearRegression

train_X_num_only = train_X_num_only_df.to_numpy()
train_y_num_only = train_y_num_only_df.to_numpy()

reg = LinearRegression().fit(train_X_num_only, train_y_num_only)

In [43]:
# R-squared on the line was fit to

reg.score(train_X_num_only, train_y_num_only)

0.8021913655309848

In [44]:
# R-squared on the held out data

test_X_num_only = test_X_num_only_df.to_numpy()
test_y_num_only = test_y_num_only_df.to_numpy()

reg.score(test_X_num_only, test_y_num_only)

0.8236218000546741

In [45]:
reg.coef_

array([-1.40753454e+00, -1.63037224e+02,  4.02508103e-01,  1.79122706e+04,
        4.78476723e+03,  3.88727262e+02,  8.70115566e+01,  8.43571013e+00,
       -3.30554191e+00,  3.45178443e+00,  8.58195265e+00,  1.78339408e+01,
        1.97956700e+01, -1.11930593e+01,  2.64365515e+01,  1.22959330e+04,
        3.28055979e+02,  4.63755817e+03, -2.87093634e+03, -8.30662580e+03,
       -1.33658793e+04,  4.87583348e+03,  4.32263819e+03,  1.22947767e+04,
       -2.53739789e+00,  3.22795004e+01, -4.06929138e+00,  1.88982904e+01,
        4.95394469e+01,  4.94219874e+01, -2.75798068e+01,  5.80217504e-02,
        1.03115189e+02, -5.87140050e+02])